# **Workshop**: Automated Text Analysis

## **Instructor:** [Felipe Lamarca (IESP-UERJ)](https://felipelamarca.com/)

Neste notebook, vamos ver como utilizar alguns modelos disponíveis no HuggingFace. Vamos começar fazendo o download das bibliotecas necessárias:

In [ ]:
!pip install -U sentence-transformers transformers datasets huggingface_hub fsspec

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datasets import load_dataset

Vamos fazer o download de um modelo. Em especial, esse é um modelo de linguagem pequeno e pré-treinado com dados em português.

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Agora, vamos computar os embeddings de cada uma das frases abaixo. Lembre-se: embeddings são representações numéricas de palavras ou frases e, graças aos transformers, carregam características semânticas.

## **Embeddings**

In [ ]:
sentences = [
    "O Banco Central aumentou a taxa de juros.",
    "Sentei-me num banco",
    "O Flamengo vencerá contra o Bayern."
]
embeddings = model.encode(sentences, normalize_embeddings=True)

In [ ]:
sim = cosine_similarity(embeddings)
print(np.round(sim, 2))

## **Análise de sentimento**

In [ ]:
sentiment = pipeline(
    "text-classification",
    model="pysentimiento/bertweet-pt-sentiment",
    top_k=None,              # devolve scores para POS / NEG / NEU
    truncation=True
)

In [ ]:
sentiment("Esse celular é ótimo!")

In [ ]:
sentiment("A comida que eu comi estava horrível.")

In [ ]:
sentiment("Não tenho certeza sobre o que estou sentindo...")

## **Geração automática de resumos**

https://huggingface.co/datasets/UFRGS/brwac

In [ ]:
summarizer = pipeline(
    "summarization",
    model="recogna-nlp/ptt5-base-summ-cstnews",   # T5 pré-treinado no BrWaC + fine-tuning CSTNews
    max_new_tokens=80, min_new_tokens=15
)

In [ ]:
texto = """
A Praia de Copacabana já perdeu 10% da faixa de areia nos últimos dez anos. E, se o planeta continuar aquecendo no ritmo atual, o futuro da orla carioca é sombrio. Segundo estudo da Universidade Federal do Rio de Janeiro (UFRJ), praias como Leme, Copacabana, Ipanema e Leblon podem recuar de 70 a 100 metros até 2100, transformando-se em filetes de areia. Os dados fazem parte de uma projeção baseada em modelos climáticos internacionais e foram publicados na revista Natural Hazards. As informações são do jornal O Globo.
O levantamento analisou o impacto da elevação do nível do mar em toda a faixa entre o Porto do Rio e o Leblon, incluindo a Baía de Guanabara, a Lagoa Rodrigo de Freitas e áreas vulneráveis como os manguezais de Guapimirim. Em Botafogo, por exemplo, a perda estimada é de até 70 metros. Em Niterói, o estudo avaliou desde a região portuária até a praia e a lagoa de Piratininga.
“O nível do mar mais elevado impedirá que a água de grandes chuvas escoe das áreas mais baixas, como a Lagoa e o entorno da Baía de Guanabara”, explicou a pesquisadora Raquel Toste, do Laboratório de Métodos Computacionais (Lamce) da Coppe/UFRJ, que assina o estudo.
"""

print(summarizer(texto)[0]["summary_text"])

## **Conversação e prompting**

In [ ]:
# Carregando o modelo ajustado para o português
generator = pipeline(
    "text-generation",
    model="rhaymison/phi-3-portuguese-tom-cat-4k-instruct",
    max_new_tokens=100,
    temperature=0.7,
)

In [ ]:
# Escrevendo um prompt simples
prompt = (
    "<s><|system|>\nVocê é um assistente inteligente e gentil.\n"
    "<|user|>\nExplique o que é aprendizado de máquina em termos simples.\n"
    "<|assistant|>\n"
)

# Gerando resposta
resposta = generator(prompt)[0]["generated_text"]

# Imprimindo apenas a resposta do assistente
print(resposta.split("<|assistant|>")[-1].strip())

## **Exercício**

**Parte 1:** use o modelo `pierreguillou/t5-base-pt-zero-shot` para prever o tema de um artigo sem treino supervisionado;

**Parte 2:** Encontre artigos semelhantes com base em embeddings de sentenças. Para gerar os embeddings, use o modelo `tgsc/sentence-transformer-ult5-pt-small`;

**Parte 3:** Gere resumos para os artigos com o modelo `recogna-nlp/ptt5-base-summ-cstnews`.

In [ ]:
from datasets import load_dataset
ds = load_dataset("iara-project/news-articles-ptbr-dataset", split="test").shuffle(seed=42).select(range(100))